<a href="https://colab.research.google.com/github/AkashParua/Object_detection_algorithms/blob/main/OptimizedYOLONAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install super-gradients

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 61.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 114.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0

In [7]:
import cv2
from super_gradients.training import models
import torch
# Create a background subtractor object
bg_subtractor = cv2.createBackgroundSubtractorMOG2()
yolo_nas_l = models.get("yolo_nas_s", pretrained_weights="coco")
model = yolo_nas_l.to("cuda" if torch.cuda.is_available() else "cpu")
# Open a video capture object
frame = cv2.imread('/content/0001TP_009210.png')   # Replace with your video file path
# Apply background subtraction
fg_mask = bg_subtractor.apply(frame)

# Apply morphological operations to enhance the mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)

# Find contours in the foreground mask
contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate over the contours and draw bounding boxes
for contour in contours:
    # Filter contours based on area or other criteria if needed
    if cv2.contourArea(contour) > 100:
        x, y, w, h = cv2.boundingRect(contour)
        roi = frame[y:y+h, x:x+w]
        out = model.predict(roi)
        prediction_objects = list(out._images_prediction_lst)[0]
        bboxes = prediction_objects.prediction.bboxes_xyxy
        int_labels = prediction_objects.prediction.labels.astype(int)
        class_names = prediction_objects.class_names
        nm = [class_names[i] for i in int_labels]
        conf = prediction_objects.prediction.confidence.astype(float)
        for i,bbox in enumerate(bboxes):
            x1 = bbox[0]
            y1 = bbox[1]
            w1 = bbox[2] - x1
            h1 = bbox[3] - y1
            print(type(x1))
            print(x1,y1,w1,h1)
            cv2.rectangle(frame, (int(x1) , int(y1)), (int(x1 + w1) ,int(y1 + h1)), (0, 255, 0), 2)


# Display the frame with bounding boxes
cv2.imwrite('output.png', frame)

[2023-06-16 11:22:12] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-06-16 11:22:13] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


True

<class 'numpy.float32'>
51.792454 371.13208 85.636795 239.3396
<class 'numpy.float32'>
394.52832 363.39624 344.05658 332.73584
<class 'numpy.float32'>
326.86322 435.61322 20.97876 74.8349
<class 'numpy.float32'>
688.2076 416.32077 95.14148 149.38681
<class 'numpy.float32'>
369.74646 394.32785 9.857025 31.61554
<class 'numpy.float32'>
353.99176 440.4717 16.80719 55.49527
<class 'numpy.float32'>
394.2453 362.73584 343.86792 335.37738
<class 'numpy.float32'>
290.6427 429.88208 20.99646 76.4151
<class 'numpy.float32'>
687.3585 415.3302 96.27356 149.85846
<class 'numpy.float32'>
315.20636 435.25943 13.58197 66.4151
<class 'numpy.float32'>
402.02832 311.65094 108.34906 158.1604
<class 'numpy.float32'>
342.3762 438.9387 13.123535 56.17923
<class 'numpy.float32'>
346.1262 438.18396 11.32666 57.79483
<class 'numpy.float32'>
369.65802 441.0849 16.49176 55.90213
<class 'numpy.float32'>
349.86145 436.98114 17.50885 59.44574
<class 'numpy.float32'>
340.97287 437.94223 13.01297 65.92569
<class 'nump